<a href="https://colab.research.google.com/github/2403a52269-code/NLP/blob/main/NLP_6_3_2403A52269.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. LDA with Sample Data and BoW**

In [1]:
import pandas as pd
data=pd.read_excel('/content/LDA-Data.xlsx')
data.head()

,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import re
def nltk_preprocessing_pipeline(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)

    text = text.lower()
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokenized_words = word_tokenize(text)
    filtered_words = [word for word in tokenized_words if word not in stop_words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    clean_summary = ' '.join(lemmatized_words)

    return clean_summary


In [5]:
data['clean_News'] = data['News'].apply(nltk_preprocessing_pipeline)
print("\nprevious clean_summaries and new clean_summaries_pipeline comparision:")
print(data[['clean_News']].head())


previous clean_summaries and new clean_summaries_pipeline comparision:
                       clean_News
0      virat scored century match
1                    bjp election
2       bumra took 5 wicket match
3  congress form state government


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_df=0.95, min_df=1, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(data['clean_News'])

In [7]:
feature_names = count_vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(doc_term_matrix.toarray(),columns=feature_names)
bow_top_10 = bow_df.head(10)
print(bow_top_10)

   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


In [8]:
from sklearn.decomposition import LatentDirichletAllocation
num_topics = 2
LDA = LatentDirichletAllocation(n_components=num_topics, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=2, random_state=42)

In [9]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx in range(len(model.components_)):
        print(f"\nTopic {topic_idx}:")
        topic_weights = model.components_[topic_idx]
        sorted_indices = topic_weights.argsort()[::-1]
        top_indices = sorted_indices[:num_top_words]
        for idx in top_indices:
            print(feature_names[idx], end=" ")
        print()

In [10]:
num_top_words = 10
print(f"\nTop {num_top_words} words per topic:")
display_topics(LDA, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic:

Topic 0:
form government congress state election bjp match wicket bumra took 

Topic 1:
match virat century scored took bumra wicket bjp election state 


In [11]:
document_topics = LDA.transform(doc_term_matrix)
data['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(data[['clean_News', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                       clean_News  topic
0      virat scored century match      1
1                    bjp election      0
2       bumra took 5 wicket match      1
3  congress form state government      0


# **2. LDA with Kaggle Data and BoW**

In [12]:
import pandas as pd

In [17]:
df = pd.read_csv('arxiv_data.csv', engine='python', nrows=1000)
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [33]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def nltk_preprocessing_pipeline(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = text.lower()

    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokenized_words = word_tokenize(text)

    filtered_words = [word for word in tokenized_words if word not in stop_words]

    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    clean_summary = ' '.join(lemmatized_words)

    return clean_summary

print("NLTK preprocessing pipeline function created successfully!")

NLTK preprocessing pipeline function created successfully!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
df['clean_summaries_pipeline'] = df['summaries'].apply(nltk_preprocessing_pipeline)
print("\nComparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):")
print(df[['clean_summaries_pipeline']].head())


Comparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):
                            clean_summaries_pipeline
0  stereo matching one widely used technique infe...
1  recent advancement artificial intelligence ai ...
2  paper proposed novel mutual consistency networ...
3  consistency training proven advanced semisuper...
4  ensure safety automated driving correct percep...


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(df['clean_summaries_pipeline'])

In [23]:
feature_names = count_vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(doc_term_matrix.toarray(),columns=feature_names)
bow_top_10 = bow_df.head(10)
print(bow_top_10)

   01  011  014  049  059  060  065  084  089  091  ...  xray  xrays  year  \
0   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
1   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
2   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
3   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
4   0    0    0    0    0    0    0    0    0    0  ...     0      0     1   
5   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
6   0    0    0    0    0    0    0    0    0    0  ...     0      0     1   
7   0    0    0    0    0    0    0    0    0    0  ...     1      0     0   
8   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
9   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   

   yes  yield  yielded  yielding  youtube  youtubevos  zurich  
0    0      0        0         0        0           0       0  
1    0      0

In [24]:
from sklearn.decomposition import LatentDirichletAllocation
num_topics = 2
LDA = LatentDirichletAllocation(n_components=num_topics, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=2, random_state=42)

In [25]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx in range(len(model.components_)):
        print(f"\nTopic {topic_idx}:")
        topic_weights = model.components_[topic_idx]
        sorted_indices = topic_weights.argsort()[::-1]
        top_indices = sorted_indices[:num_top_words]
        for idx in top_indices:
            print(feature_names[idx], end=" ")
        print()

In [26]:
num_top_words = 10
print(f"\nTop {num_top_words} words per topic:")
display_topics(LDA, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic:

Topic 0:
method network feature model proposed approach result algorithm based semantic 

Topic 1:
network method model learning data deep training medical task performance 


In [27]:
document_topics = LDA.transform(doc_term_matrix)
df['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(df[['clean_summaries_pipeline', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                            clean_summaries_pipeline  topic
0  stereo matching one widely used technique infe...      1
1  recent advancement artificial intelligence ai ...      1
2  paper proposed novel mutual consistency networ...      1
3  consistency training proven advanced semisuper...      1
4  ensure safety automated driving correct percep...      1


# **3. NMF with Sample Data and BoW**

In [34]:
import pandas as pd
data=pd.read_excel('/content/LDA-Data.xlsx')
data.head()

,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


In [35]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [36]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [37]:
import re
def nltk_preprocessing_pipeline(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)

    text = text.lower()
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokenized_words = word_tokenize(text)
    filtered_words = [word for word in tokenized_words if word not in stop_words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    clean_summary = ' '.join(lemmatized_words)

    return clean_summary

In [38]:
data['clean_News'] = data['News'].apply(nltk_preprocessing_pipeline)
print("\nprevious clean_summaries and new clean_summaries_pipeline comparision:")
print(data[['clean_News']].head())


previous clean_summaries and new clean_summaries_pipeline comparision:
                       clean_News
0      virat scored century match
1                    bjp election
2       bumra took 5 wicket match
3  congress form state government


In [39]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_df=0.95, min_df=1, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(data['clean_News'])

In [40]:
feature_names = count_vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(doc_term_matrix.toarray(),columns=feature_names)
bow_top_10 = bow_df.head(10)
print(bow_top_10)

   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


In [41]:
from sklearn.decomposition import NMF
num_topics = 2
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(doc_term_matrix)
print("NMF model initialized and fitted successfully.")

NMF model initialized and fitted successfully.


In [42]:
print(f"\nTop {num_top_words} words per topic (NMF):")
display_topics(nmf_model, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic (NMF):

Topic 0:
match virat took scored wicket bumra century bjp election state 

Topic 1:
state form congress government election bjp took virat wicket scored 


In [43]:
document_topics = nmf_model.transform(doc_term_matrix)
data['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(data[['clean_News', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                       clean_News  topic
0      virat scored century match      0
1                    bjp election      1
2       bumra took 5 wicket match      0
3  congress form state government      1


# **4. NMF with Kaggle Data and BoW**

In [44]:
import pandas as pd

In [45]:
df = pd.read_csv('arxiv_data.csv', engine='python', nrows=1000)
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [46]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def nltk_preprocessing_pipeline(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = text.lower()

    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokenized_words = word_tokenize(text)

    filtered_words = [word for word in tokenized_words if word not in stop_words]

    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    clean_summary = ' '.join(lemmatized_words)

    return clean_summary

print("NLTK preprocessing pipeline function created successfully!")

NLTK preprocessing pipeline function created successfully!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [47]:
df['clean_summaries_pipeline'] = df['summaries'].apply(nltk_preprocessing_pipeline)
print("\nComparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):")
print(df[['clean_summaries_pipeline']].head())


Comparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):
                            clean_summaries_pipeline
0  stereo matching one widely used technique infe...
1  recent advancement artificial intelligence ai ...
2  paper proposed novel mutual consistency networ...
3  consistency training proven advanced semisuper...
4  ensure safety automated driving correct percep...


In [48]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(df['clean_summaries_pipeline'])

In [49]:
feature_names = count_vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(doc_term_matrix.toarray(),columns=feature_names)
bow_top_10 = bow_df.head(10)
print(bow_top_10)

   01  011  014  049  059  060  065  084  089  091  ...  xray  xrays  year  \
0   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
1   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
2   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
3   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
4   0    0    0    0    0    0    0    0    0    0  ...     0      0     1   
5   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
6   0    0    0    0    0    0    0    0    0    0  ...     0      0     1   
7   0    0    0    0    0    0    0    0    0    0  ...     1      0     0   
8   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   
9   0    0    0    0    0    0    0    0    0    0  ...     0      0     0   

   yes  yield  yielded  yielding  youtube  youtubevos  zurich  
0    0      0        0         0        0           0       0  
1    0      0

In [29]:
from sklearn.decomposition import NMF
num_topics = 2
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(doc_term_matrix)
print("NMF model initialized and fitted successfully.")

NMF model initialized and fitted successfully.


In [30]:
num_top_words = 10
print(f"\nTop {num_top_words} words per topic (NMF):")
display_topics(nmf_model, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic (NMF):

Topic 0:
method model learning data training medical deep domain approach performance 

Topic 1:
network neural architecture feature task convolutional deep proposed propose performance 


In [32]:
document_topics = nmf_model.transform(doc_term_matrix)
df['topic'] = document_topics.argmax(axis=1)

print("\n 5 first rows in DataFrame with assigned topics:")
print(df[['clean_summaries_pipeline', 'topic']].head())


 5 first rows in DataFrame with assigned topics:
                            clean_summaries_pipeline  topic
0  stereo matching one widely used technique infe...      1
1  recent advancement artificial intelligence ai ...      0
2  paper proposed novel mutual consistency networ...      0
3  consistency training proven advanced semisuper...      0
4  ensure safety automated driving correct percep...      0


# **5. NMF with Sample Data and TFIDF**

In [50]:
import pandas as pd

In [51]:
data=pd.read_excel('/content/LDA-Data.xlsx')
data.head()

,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


In [56]:
data['clean_News'] = data['News'].apply(nltk_preprocessing_pipeline)

print(data[['clean_News']].head())

                       clean_News
0      virat scored century match
1                    bjp election
2       bumra took 5 wicket match
3  congress form state government


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1, stop_words='english')

doc_term_matrix = tfidf_vectorizer.fit_transform(data['clean_News'])

In [58]:
feature_names = tfidf_vectorizer.get_feature_names_out()

tfidf_df = pd.DataFrame(doc_term_matrix.toarray(), columns=feature_names)

print(tfidf_df.head())

        bjp     bumra   century  congress  election  form  government  \
0  0.000000  0.000000  0.525473       0.0  0.000000   0.0         0.0   
1  0.707107  0.000000  0.000000       0.0  0.707107   0.0         0.0   
2  0.000000  0.525473  0.000000       0.0  0.000000   0.0         0.0   
3  0.000000  0.000000  0.000000       0.5  0.000000   0.5         0.5   

      match    scored  state      took     virat    wicket  
0  0.414289  0.525473    0.0  0.000000  0.525473  0.000000  
1  0.000000  0.000000    0.0  0.000000  0.000000  0.000000  
2  0.414289  0.000000    0.0  0.525473  0.000000  0.525473  
3  0.000000  0.000000    0.5  0.000000  0.000000  0.000000  


In [59]:
from sklearn.decomposition import NMF

num_topics = 2

nmf_model = NMF(n_components=num_topics, random_state=42)

nmf_model.fit(doc_term_matrix)

print("NMF model fitted successfully using TF-IDF.")

NMF model fitted successfully using TF-IDF.


In [60]:
num_top_words = 10

print(f"\nTop {num_top_words} words per topic (NMF + TFIDF):")

display_topics(nmf_model, feature_names, num_top_words)


Top 10 words per topic (NMF + TFIDF):

Topic 0:
match bumra wicket took virat scored century state government election 

Topic 1:
election bjp form government state congress took virat wicket scored 


In [61]:
document_topics = nmf_model.transform(doc_term_matrix)

data['topic'] = document_topics.argmax(axis=1)

print(data[['clean_News', 'topic']])

                       clean_News  topic
0      virat scored century match      0
1                    bjp election      1
2       bumra took 5 wicket match      0
3  congress form state government      1


# **6. NMF with Kaggle Data and TFIDF**

In [62]:
import pandas as pd

In [70]:
df = pd.read_csv('arxiv_data.csv', engine='python', nrows=1000)
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [64]:
df['clean_summaries_pipeline'] = df['summaries'].apply(nltk_preprocessing_pipeline)

print(df[['clean_summaries_pipeline']].head())

                            clean_summaries_pipeline
0  stereo matching one widely used technique infe...
1  recent advancement artificial intelligence ai ...
2  paper proposed novel mutual consistency networ...
3  consistency training proven advanced semisuper...
4  ensure safety automated driving correct percep...


In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vectorizer.fit_transform(df['clean_summaries_pipeline'])

In [66]:
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(doc_term_matrix.toarray(), columns=feature_names)
print(tfidf_df.head())

    01  011  014  049  059  060  065  084  089  091  ...  xray  xrays  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0    0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0    0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0    0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0    0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0    0.0   

       year  yes  yield  yielded  yielding  youtube  youtubevos  zurich  
0  0.000000  0.0    0.0      0.0       0.0      0.0         0.0     0.0  
1  0.000000  0.0    0.0      0.0       0.0      0.0         0.0     0.0  
2  0.000000  0.0    0.0      0.0       0.0      0.0         0.0     0.0  
3  0.000000  0.0    0.0      0.0       0.0      0.0         0.0     0.0  
4  0.055707  0.0    0.0      0.0       0.0      0.0         0.0     0.0  

[5 rows x 4386 columns]


In [67]:
from sklearn.decomposition import NMF

num_topics = 2
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(doc_term_matrix)
print("NMF model fitted successfully using TF-IDF.")

NMF model fitted successfully using TF-IDF.


In [68]:
num_top_words = 10
print(f"\nTop {num_top_words} words per topic (NMF + TFIDF):")
display_topics(nmf_model, feature_names, num_top_words)


Top 10 words per topic (NMF + TFIDF):

Topic 0:
network method feature architecture model object proposed result approach 3d 

Topic 1:
domain data learning annotation training model label medical method labeled 


In [69]:
document_topics = nmf_model.transform(doc_term_matrix)
df['topic'] = document_topics.argmax(axis=1)
print(df[['clean_summaries_pipeline', 'topic']].head())

                            clean_summaries_pipeline  topic
0  stereo matching one widely used technique infe...      0
1  recent advancement artificial intelligence ai ...      1
2  paper proposed novel mutual consistency networ...      1
3  consistency training proven advanced semisuper...      1
4  ensure safety automated driving correct percep...      0
